In [ ]:
import bezier
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import networkx as nx
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import itertools
import functools
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
%load_ext autoreload
%autoreload 2

In [ ]:
class GraphGram(object):
    
    def __init__(self, graph, xoff=0, yoff=0, scale=1, layout_method='kamada_kawai_layout'):
        self.graph = graph
        self._nodes = None
        self.xoff = xoff
        self.yoff = yoff
        self.scale = scale
        self.layout_method = layout_method
    
    @property
    def center(self):
        return np.array((self.xoff, self.yoff))
    
    @property
    def edges(self):
        return list(self.graph.edges)
    
    @property
    def layout_function(self):
        try:
            f = getattr(nx.layout, self.layout_method)
        except AttributeError:
            layout_functions = [a for a in dir(nx.layout) if 'layout' in a]
            error_string = f'''{self.layout_method} not found in networkx.layout module; 

choose from {layout_functions}
            '''
            print(error_string)
        return f
    
    @functools.lru_cache
    def get_layout(self, *args, **kwargs):
        self._nodes = self.layout_function(
            self.graph,
            scale=self.scale,
            center=self.center,
            *args, **kwargs)
    
    @property
    def nodes(self):
        if self._nodes is None:
            self.get_layout()
        return self._nodes
    
    @property
    def node_pts(self):
        return {k:Point(xy) for k, xy in self.nodes.items()}
    
    @property
    def pts(self):
        return MultiPoint([p for p in self.node_pts.values()])
    
    @property
    def lines(self):
        lines = []
        for n0,n1 in self.edges:
            p0 = self.node_pts[n0]
            p1 = self.node_pts[n1]
            lines.append(LineString([p0, p1]))
        return MultiLineString(lines)    
        

# strogatz

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=20
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
DEGREE = 13
SCALE = 15
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=40, ystep=40, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0.2, 1.] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 3
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='kamada_kawai_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    new_row = row.to_dict()
    new_row['geometry'] = gg.lines
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance=0.5
sk.vpype(f'linesort')

sk.display()

In [ ]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0149_graph_test.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# strogatz

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=18
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
DEGREE = 33
SCALE = 8
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=20, ystep=20, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

In [ ]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# binary tree

In [ ]:
# make page
paper_size = '11x17 inches'
border:float=18
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
DEGREE = 33
SCALE = 20
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=50, ystep=50, flatmesh=True)

In [ ]:
r_gen = lambda x: int(np.interp(x, [xs.min(), xs.max()], [2, 8] ))
_r_gen = gp.make_callable(r_gen)

In [ ]:
k_gen = 3
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
nx.number_of_nonisomorphic_trees(10)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spectral_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

In [ ]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)

# strogatz

In [ ]:
# make page
paper_size = '23.42x16.92 inches'
border:float=35
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
DEGREE = 33
SCALE = 8
(xbins, ybins), (xs, ys) = gp.overlay_grid(drawbox, xstep=20, ystep=20, flatmesh=True)

In [ ]:
p_gen = lambda x: np.interp(x, [xs.min(), xs.max()], [0., 0.6] )
_p_gen = gp.make_callable(p_gen)

In [ ]:
k_gen = 2
_k_gen = gp.make_callable(k_gen)

In [ ]:
df = pd.DataFrame({
    'x':xs,
    'y':ys,
    'k':_k_gen(xs),
    'p':_p_gen(xs)
})
df['k'] = df['k'].astype(int)

In [ ]:
new_rows = []
for i, row in df.iterrows():
    k = row['k'].astype(int)
    G = nx.connected_watts_strogatz_graph(n=DEGREE, k=k, p=row['p'])
    gg = GraphGram(graph=G, layout_method='spring_layout',
                   xoff=row['x'], yoff=row['y'], scale=SCALE)
    
    bezs = []
    for ls in gg.lines:
        bez = gp.LineString_to_jittered_bezier(
            ls, xstd=0., ystd=0., normalized=True, n_eval_points=2)
        bezs.append(bez)
    bezs = gp.merge_LineStrings(bezs)
    new_row = row.to_dict()
    new_row['geometry'] = bezs
    new_rows.append(new_row)
    
gdf = geopandas.GeoDataFrame(new_rows)
layers = []
layers.append(gp.merge_LineStrings(gdf.geometry))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

tolerance= 0.5
sk.vpype(f'linemerge --tolerance {tolerance}mm linesort')

sk.display()

In [ ]:
savedir='/mnt/c/code/side/plotter_images/oned_outputs'
filename = '0151_strogatz_graphs.svg'

In [ ]:
savepath = Path(savedir).joinpath(filename).as_posix()
sk.save(savepath)